In [64]:
import pandas as pd

In [18]:
years = [2022, 2023]

#Россия
all_dataRFPL = pd.DataFrame()
for year in years:
    rfpl_path = f'elo/csv/Russia/{year}/RFPL.csv'
    kubok_path = f'elo/csv/Russia/{year}/KUBOK.csv'

    data_RFPL = pd.read_csv(rfpl_path)
    data_kubok = pd.read_csv(kubok_path)

    unique_teams_RFPL = data_RFPL['home_team_id'].unique()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_RFPL) & data_kubok['away_team_id'].isin(unique_teams_RFPL)]
    all_dataRFPL = pd.concat([all_dataRFPL, data_kubok, data_RFPL])

In [19]:
all_dataRFPL = all_dataRFPL.sort_values('timestamp')
all_dataRFPL.reset_index(drop=True, inplace=True)


In [22]:
#Англия
all_dataPL = pd.DataFrame()
for year in years:
    pl_path = f'elo/csv/England/{year}/PL.csv'
    kubok_path = f'elo/csv/England/{year}/cup.csv'
    carabao_path = f'elo/csv/England/{year}/carabao.csv'

    data_PL = pd.read_csv(pl_path)
    data_kubok = pd.read_csv(kubok_path)
    data_carabao = pd.read_csv(carabao_path)

    unique_teams_PL = data_PL['home_team_id'].unique()

    data_kubok = data_kubok[data_kubok['home_team_id'].isin(unique_teams_PL) & data_kubok['away_team_id'].isin(unique_teams_PL)]
    data_carabao = data_carabao[data_carabao['home_team_id'].isin(unique_teams_PL) & data_carabao['away_team_id'].isin(unique_teams_PL)]
    all_dataPL = pd.concat([all_dataPL, data_kubok, data_PL, data_carabao])

all_dataPL = all_dataPL.sort_values('timestamp')
all_dataPL.reset_index(drop=True, inplace=True)
all_dataPL

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,867946,1659726000,52,Crystal Palace,0.0,2.0,42,Arsenal
1,867947,1659785400,36,Fulham,2.0,2.0,40,Liverpool
2,867951,1659794400,34,Newcastle,2.0,0.0,65,Nottingham Forest
3,867949,1659794400,63,Leeds,2.0,1.0,39,Wolves
4,867948,1659794400,35,Bournemouth,2.0,0.0,66,Aston Villa
...,...,...,...,...,...,...,...,...
492,1035110,1696696200,52,Crystal Palace,0.0,0.0,65,Nottingham Forest
493,1035115,1696770000,48,West Ham,2.0,2.0,34,Newcastle
494,1035116,1696770000,39,Wolves,1.0,1.0,66,Aston Villa
495,1035108,1696770000,51,Brighton,2.0,2.0,40,Liverpool


In [24]:
all_data = pd.concat([all_dataPL, all_dataRFPL]).sort_values('timestamp')
all_data.reset_index(drop=True, inplace=True)

In [50]:
all_data

,fixtueID,timestamp,home_team_id,home_team_name,home_goals,away_goals,away_team_id,away_team_name
0,879867,1657904400,1994,Khimki,1.0,1.0,596,Zenit Saint Petersburg
1,879868,1657972800,555,CSKA Moscow,2.0,0.0,1084,Ural
2,879869,1657981800,1080,Orenburg,2.0,4.0,1079,Krylya Sovetov
3,879870,1657990800,1085,Akhmat Grozny,1.0,1.0,558,Spartak Moscow
4,879871,1658059200,3985,Torpedo Moskva,1.0,3.0,2012,PFC Sochi
...,...,...,...,...,...,...,...,...
933,1035116,1696770000,39,Wolves,1.0,1.0,66,Aston Villa
934,1042096,1696771800,1088,Dinamo Moscow,0.0,0.0,1993,Fakel Voronezh
935,1035107,1696779000,42,Arsenal,1.0,0.0,50,Manchester City
936,1042094,1696780800,558,Spartak Moscow,2.0,2.0,555,CSKA Moscow


In [92]:
import http.client
import json

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-rapidapi-host': "v3.football.api-sports.io",
    'x-rapidapi-key': "55d8d0822bdb3026f52b479e85e3a447"
    }

result_data = []
for index, row in all_data.iterrows():
    fixtureID = row['fixtueID']
    home_team_id = row['home_team_id']
    home_goals = row['home_goals']
    away_team_id = row['away_team_id']
    away_goals = row['away_goals']
    
    for team_info, position in [((home_team_id, home_goals), 'home'), ((away_team_id, away_goals), 'away')]:
        conn.request("GET", f"/fixtures/statistics?fixture={fixtureID}&team={team_info[0]}", headers=headers)
        res = conn.getresponse()
        data = res.read()
        json_data = json.loads(data)

        if 'response' in json_data and json_data['response']:
            json_data = json_data['response']
            if json_data and len(json_data) > 0 and 'statistics' in json_data[0]:
                stat = json_data[0]['statistics']
            else:
                stat = []
        else:
            stat = []

        if len(stat) > 0:
            # Получение ударов в створ и XG
            shots_on_target = [item['value'] for item in stat if item['type'] == 'Shots on Goal']
            if shots_on_target and isinstance(shots_on_target[0], int):
                shots_on_target = shots_on_target[0]
            else:
                shots_on_target = team_info[1]

            xg = [item['value'] for item in stat if item['type'] == 'expected_goals']
            if xg and isinstance(xg[0], str):
                xg = xg[0]
            else:
                xg = team_info[1]

            result_data.append([fixtureID, team_info[0], position, team_info[1], shots_on_target, xg])

result_df = pd.DataFrame(result_data, columns=['fixtureID', 'team_id', 'position', 'goals', 'shots', 'xg'])

print(result_df)

    

      fixtureID  team_id position  goals  shots    xg
0        879867     1994     home    1.0    3.0   1.0
1        879867      596     away    1.0    2.0   1.0
2        879868      555     home    2.0    8.0   2.0
3        879868     1084     away    0.0    2.0   0.0
4        879869     1080     home    2.0    8.0   2.0
...         ...      ...      ...    ...    ...   ...
1799    1035107       50     away    0.0    1.0  0.48
1800    1042094      558     home    2.0    4.0   2.0
1801    1042094      555     away    2.0    5.0   2.0
1802    1042097     1085     home    0.0    4.0   0.0
1803    1042097     1083     away    1.0    6.0   1.0

[1804 rows x 6 columns]


In [99]:
result_df['xg'] = result_df['xg'].astype('float')
result_df.to_csv('match_data.csv',float_format='%.2f')